# Метод касательных (Ньютона-Рафсона) и сравнение с другими методами
# Tangent Method (Newton-Raphson) and Comparison with Other Methods

**Цель:** Изучить алгоритм метода Ньютона для нахождения корней нелинейных уравнений, реализовать его на Julia и сравнить эффективность с методом секущих и методом половинного деления.
**Objective:** To study the Newton's method algorithm for finding roots of nonlinear equations, implement it in Julia, and compare its efficiency with the Secant method and the Bisection method.

**Контекст:** В биологии и медицине поиск корней часто возникает при расчете равновесных концентраций лекарств, точек бифуркации в популяционных моделях или калибровке параметров.
**Context:** In biology and medicine, root finding often arises when calculating equilibrium drug concentrations, bifurcation points in population models, or parameter calibration.

**Терминология:** В русской литературе "Метод Рафтона" часто используется как синоним метода Ньютона или Ньютона-Рафсона. Мы будем использовать стандартное название **Newton-Raphson**.
**Terminology:** In Russian literature, "Rafton's Method" is often used as a synonym for Newton's method or Newton-Raphson. We will use the standard name **Newton-Raphson**.

In [3]:
# Установка и подключение необходимых пакетов
# Installing and loading necessary packages

# Если пакеты не установлены, раскомментируйте строку ниже
# If packages are not installed, uncomment the line below
# using Pkg; Pkg.add(["Plots", "BenchmarkTools", "Printf"])

using Plots          # Для визуализации / For visualization
using BenchmarkTools # Для замера времени / For timing measurements
using Printf         # Для форматированного вывода / For formatted output

# Настройка темы графиков для лучшей читаемости
# Setting plot theme for better readability
plotlyjs() 

┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\Александр\.julia\packages\PlotlyKaleido\hHnQW\src\PlotlyKaleido.jl:38


Plots.PlotlyJSBackend()

## Теория метода Ньютона
## Newton's Method Theory

Метод Ньютона основан на линеаризации функции с помощью ряда Тейлора.
Newton's method is based on linearizing the function using a Taylor series.

**Формула итерации / Iteration Formula:**
$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

**Преимущества / Advantages:**
1. Квадратичная сходимость (быстро) вблизи корня.
   Quadratic convergence (fast) near the root.
2. Простота реализации.
   Simple implementation.

**Недостатки / Disadvantages:**
1. Требует вычисления производной $f'(x)$.
   Requires calculation of the derivative $f'(x)$.
2. Может расходиться, если начальное приближение далеко от корня.
   May diverge if the initial guess is far from the root.
3. Проблема, если $f'(x) \approx 0$.
   Problem if $f'(x) \approx 0$.

**Альтернативы / Alternatives:**
- **Метод секущих (Secant Method):** Аппроксимирует производную разностным отношением. Не требует явной формулы производной.
  Approximates derivative with finite difference. Does not require explicit derivative formula.
- **Метод половинного деления (Bisection):** Медленнее (линейная сходимость), но гарантирует нахождение корня на отрезке.
  Slower (linear convergence), but guarantees finding a root on an interval.

In [4]:
# Задача: Расчет равновесной концентрации лекарства при нелинейном метаболизме (Михаэлиса-Ментен)
# Problem: Calculate equilibrium drug concentration with nonlinear metabolism (Michaelis-Menten)

# Уравнение баланса: Скорость введения = Скорость элиминации
# Balance Equation: Infusion Rate = Elimination Rate

# R_in - скорость инфузии (мг/ч)
# R_in - infusion rate (mg/h)
R_in = 50.0 

# V_max - максимальная скорость элиминации (мг/ч)
# V_max - maximum elimination rate (mg/h)
V_max = 100.0 

# K_m - константа Михаэлиса (мг/л)
# K_m - Michaelis constant (mg/L)
K_m = 10.0 

# Функция, корень которой мы ищем: f(C) = V_max * C / (K_m + C) - R_in = 0
# Function whose root we seek: f(C) = V_max * C / (K_m + C) - R_in = 0
function pharmacokinetic_eq(C)
    return (V_max * C) / (K_m + C) - R_in
end

# Аналитическая производная функции (необходима для метода Ньютона)
# Analytic derivative of the function (required for Newton's method)
# d/dC [ (V_max * C) / (K_m + C) ] = (V_max * K_m) / (K_m + C)^2
function pharmacokinetic_deriv(C)
    return (V_max * K_m) / ((K_m + C)^2)
end

# Точное решение для проверки погрешности (можно выразить алгебраически)
# Exact solution for error checking (can be expressed algebraically)
# R_in * (K_m + C) = V_max * C  =>  R_in * K_m = C * (V_max - R_in)
C_exact = (R_in * K_m) / (V_max - R_in)

println(@sprintf "Точное значение концентрации: %.10f мг/л" C_exact)
println(@sprintf "Exact concentration value: %.10f mg/L" C_exact)

Точное значение концентрации: 10.0000000000 мг/л
Exact concentration value: 10.0000000000 mg/L


In [5]:
# Структура для хранения истории сходимости
# Structure to store convergence history
struct ConvergenceHistory
    iterations::Int
    errors::Vector{Float64}
    solution::Float64
    converged::Bool
end

# Функция метода Ньютона
# Newton's Method Function
function newton_method(f, df, x0; tol=1e-10, max_iter=100)
    x = x0
    errors = Float64[] # Вектор для хранения погрешности на каждом шаге / Vector to store error at each step
    
    converged = false
    
    for i in 1:max_iter
        fx = f(x)
        dfx = df(x)
        
        # Проверка на деление на ноль
        # Check for division by zero
        if abs(dfx) < 1e-14
            @warn "Производная слишком мала. Остановка. / Derivative too small. Stopping."
            break
        end
        
        x_new = x - fx / dfx
        
        # Запись погрешности (разница между итерациями)
        # Record error (difference between iterations)
        push!(errors, abs(x_new - x))
        
        x = x_new
        
        # Критерий остановки
        # Stopping criterion
        if abs(fx) < tol
            converged = true
            return ConvergenceHistory(i, errors, x, converged)
        end
    end
    
    return ConvergenceHistory(max_iter, errors, x, converged)
end

# Запуск метода Ньютона
# Run Newton's Method
# Начальное приближение: 5 мг/л (должно быть положительным)
# Initial guess: 5 mg/L (must be positive)
x0_newton = 5.0 
res_newton = newton_method(pharmacokinetic_eq, pharmacokinetic_deriv, x0_newton)

println(@sprintf "\nМетод Ньютона / Newton's Method:")
println(@sprintf "Итераций / Iterations: %d" res_newton.iterations)
println(@sprintf "Решение / Solution: %.10f" res_newton.solution)
println(@sprintf "Погрешность от точного / Error from exact: %.2e" abs(res_newton.solution - C_exact))


Метод Ньютона / Newton's Method:
Итераций / Iterations: 6
Решение / Solution: 10.0000000000
Погрешность от точного / Error from exact: 0.00e+00


In [6]:
# Метод секущих (не требует производной)
# Secant Method (does not require derivative)
function secant_method(f, x0, x1; tol=1e-10, max_iter=100)
    errors = Float64[]
    for i in 1:max_iter
        fx0 = f(x0)
        fx1 = f(x1)
        
        if abs(fx1 - fx0) < 1e-14
            break
        end
        
        x2 = x1 - fx1 * (x1 - x0) / (fx1 - fx0)
        push!(errors, abs(x2 - x1))
        
        x0 = x1
        x1 = x2
        
        if abs(fx1) < tol
            return ConvergenceHistory(i, errors, x1, true)
        end
    end
    return ConvergenceHistory(max_iter, errors, x1, false)
end

# Метод половинного деления (требует интервал, содержащий корень)
# Bisection Method (requires interval containing root)
function bisection_method(f, a, b; tol=1e-10, max_iter=100)
    if f(a) * f(b) > 0
        error("Функция должна иметь разные знаки на концах интервала / Function must have different signs at interval ends")
    end
    
    errors = Float64[]
    c = a
    for i in 1:max_iter
        c = (a + b) / 2
        push!(errors, (b - a) / 2) # Оценка погрешности как половина ширины интервала
                                   # Error estimate as half interval width
        
        if abs(f(c)) < tol || (b - a) / 2 < tol
            return ConvergenceHistory(i, errors, c, true)
        end
        
        if f(a) * f(c) < 0
            b = c
        else
            a = c
        end
    end
    return ConvergenceHistory(max_iter, errors, c, false)
end

# Запуск методов сравнения
# Run comparison methods
res_secant = secant_method(pharmacokinetic_eq, 5.0, 6.0) # Два начальных приближения / Two initial guesses
res_bisection = bisection_method(pharmacokinetic_eq, 0.1, 100.0) # Интервал / Interval

println(@sprintf "\nМетод секущих / Secant Method:")
println(@sprintf "Итераций / Iterations: %d" res_secant.iterations)
println(@sprintf "Погрешность / Error: %.2e" abs(res_secant.solution - C_exact))

println(@sprintf "\nМетод половинного деления / Bisection Method:")
println(@sprintf "Итераций / Iterations: %d" res_bisection.iterations)
println(@sprintf "Погрешность / Error: %.2e" abs(res_bisection.solution - C_exact))


Метод секущих / Secant Method:
Итераций / Iterations: 7
Погрешность / Error: 0.00e+00

Метод половинного деления / Bisection Method:
Итераций / Iterations: 39
Погрешность / Error: 3.77e-11


In [7]:
# Сравнение времени выполнения (Benchmark)
# Execution time comparison (Benchmark)

# Используем BenchmarkTools для точного замера
# Use BenchmarkTools for accurate measurement
bench_newton = @benchmark newton_method(pharmacokinetic_eq, pharmacokinetic_deriv, 5.0)
bench_secant = @benchmark secant_method(pharmacokinetic_eq, 5.0, 6.0)
bench_bisection = @benchmark bisection_method(pharmacokinetic_eq, 0.1, 100.0)

println("\n--- Бенчмарки (время в наносекундах) / Benchmarks (time in nanoseconds) ---")
println(@sprintf "Ньютон / Newton:   %.2f ns" minimum(bench_newton.times))
println(@sprintf "Секущих / Secant:  %.2f ns" minimum(bench_secant.times))
println(@sprintf "Деления / Bisection: %.2f ns" minimum(bench_bisection.times))

# Визуализация сходимости (Логарифмическая шкала погрешности)
# Convergence Visualization (Logarithmic error scale)
plot(title="Сходимость методов / Method Convergence",
     xlabel="Итерация / Iteration",
     ylabel="log10(Погрешность) / log10(Error)",
     legend=:topright,
     ylim=(-16, 2))

# Обрезаем массивы ошибок до минимальной длины для построения графика
# Trim error arrays to minimum length for plotting
min_len = min(length(res_newton.errors), length(res_secant.errors), length(res_bisection.errors))

plot!(1:min_len, log10.(res_newton.errors[1:min_len]), 
      label="Ньютон / Newton", linewidth=3, marker=:circle)
plot!(1:min_len, log10.(res_secant.errors[1:min_len]), 
      label="Секущих / Secant", linewidth=3, marker=:square)
plot!(1:min_len, log10.(res_bisection.errors[1:min_len]), 
      label="Деления / Bisection", linewidth=3, marker=:diamond)

# Сохранение графика (опционально)
# Save plot (optional)
# savefig("convergence_comparison.png")


--- Бенчмарки (время в наносекундах) / Benchmarks (time in nanoseconds) ---
Ньютон / Newton:   3500.00 ns
Секущих / Secant:  4828.57 ns
Деления / Bisection: 21200.00 ns


┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\Александр\.julia\packages\PlotlyKaleido\hHnQW\src\PlotlyKaleido.jl:38
┌ Warning: Kaleido startup failed with code 1.
└ @ PlotlyKaleido C:\Users\Александр\.julia\packages\PlotlyKaleido\hHnQW\src\PlotlyKaleido.jl:38


## Анализ результатов
## Results Analysis

### 1. Скорость сходимости (Convergence Speed)
- **Метод Ньютона:** Демонстрирует **квадратичную сходимость**. Обратите внимание на график: количество верных знаков удваивается на каждой итерации (крутой спуск линии). Обычно требуется 4-6 итераций для машинной точности.
  **Newton's Method:** Demonstrates **quadratic convergence**. Notice the plot: the number of correct digits doubles each iteration (steep line descent). Usually requires 4-6 iterations for machine precision.
  
- **Метод секущих:** Сходимость **сверхлинейная** (порядок ~1.618). Работает чуть медленнее Ньютона, но не требует вычисления производной, что может быть выгодно, если производная сложная.
  **Secant Method:** **Superlinear convergence** (order ~1.618). Works slightly slower than Newton, but does not require derivative calculation, which can be beneficial if the derivative is complex.

- **Метод половинного деления:** **Линейная сходимость**. Требуется значительно больше итераций (часто 30+), но метод надежен.
  **Bisection Method:** **Linear convergence**. Requires significantly more iterations (often 30+), but the method is robust.

### 2. Вычислительная стоимость (Computational Cost)
В данном примере функция простая, поэтому время выполнения в наносекундах сопоставимо. Однако, если вычисление `f(x)` или `f'(x)` требует решения сложных систем (например, в биофизике мембран), метод Ньютона может стать дороже на одну итерацию, но выиграть за счет меньшего количества итераций.
In this example, the function is simple, so execution time in nanoseconds is comparable. However, if computing `f(x)` or `f'(x)` requires solving complex systems (e.g., in membrane biophysics), Newton's method may be more expensive per iteration but win due to fewer iterations.

### 3. Рекомендации для биомоделирования
### Recommendations for Bi modeling
- Используйте **Метод Ньютона** для задач, где производная известна аналитически (как в кинетике Михаэлиса-Ментен).
  Use **Newton's Method** for tasks where the derivative is known analytically (as in Michaelis-Menten kinetics).
- Используйте **Метод секущих**, если модель задана "черным ящиком" (сложный симулятор) и производную получить трудно.
  Use **Secant Method** if the model is a "black box" (complex simulator) and derivative is hard to obtain.
- Используйте **Метод деления** для глобального поиска или когда нужна гарантия сходимости (например, при подборе параметров для клинических данных).
  Use **Bisection Method** for global search or when convergence guarantee is needed (e.g., when fitting parameters for clinical data).